In [1]:
import numpy as np
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

def mid(npArr):
    return npArr.sum() / len(npArr)

#Расчет косвенной погрешности, если величина - произведение и частное данных
def sigma_mulls(val, sigsArr, parmsArr):
    sum = 0
    for i in range(len(sigsArr)):
        sum += (sigsArr[i] / parmsArr[i])**2
    return val * (sum)**0.5


#Расчет случайной погрешности среднего
def sigma_rand(valsArr):
    N = len(valsArr)
    valMid = sum(valsArr) / N
    msum = 0
    for val in valsArr:
        msum += (val - valMid)**2
        
    return (msum / (N * (N - 1))) ** 0.5


#MNK если y = kx
##DEPRECATED
def k_only(x, y):
    return mid(x * y) / mid(x * x)

##DEPRECATED
def k_only_sigma(x, y):
    return ((mid(x * x) * mid(y * y) - mid(x * y)**2) / (len(x) * mid(x * x)**2))**0.5

#MNK для y = kx + a
def k_coef(x, y):
    return (mid(x * y) - mid(x) * mid(y)) / (mid(x * x) - mid(x)**2)

def b_coef(x, y):
    return mid(y) - k_coef(x, y) * mid(x)

def k_sigma(x, y):
    return ((mid(y * y) - mid(y)**2) / (mid(x * x) - mid(x)**2) - k_coef(x, y)**2)**0.5  / len(x)**0.5

def b_sigma(x, y):
    return k_sigma(x, y) * (mid(x**2) - mid(x)**2)**0.5


In [26]:
sigPl = 0.125
sigI = 2.5 / 2 / 100
sigUl = 0.5
Omeg = 50 #Herz

def sigL(ul, Pl, I):
    sigA = -Pl / I**2 / Omeg / (ul**2 - (Pl / I)**2)**0.5
    sigB = Pl**2 / I**3 / Omeg / (ul**2 - (Pl / I)**2)**0.5
    sigC = ul / Omeg / (ul**2 - (Pl / I)**2)**0.5   
    return ((sigA * sigPl)**2 + (sigB * sigI)**2 + (sigC * sigUl)**2)**0.5

def sigRl(Pl, I):
    return Pl / I**2 * ((sigPl / Pl)**2 + (2 * sigI / I)**2)**0.5

uls = np.array([103., 89., 78., 70., 64., 60., 55.])
Is = np.array([20., 29., 33., 36., 38., 39., 40.]) * 2.5 / 100
Ps = np.array([66., 53., 46., 43., 40., 38., 36.]) / 4 #Watt

Ls = np.array([])
Rs = np.array([])
N = 7
for i in range(N):
    print("#{}  rl: {}  L: {}".format(i + 1, Ps[i] / Is[i]**2, (uls[i]**2 - (Ps[i] / Is[i])**2)**0.5 / Omeg))
    Ls = np.append(Ls, (uls[i]**2 - (Ps[i] / Is[i])**2)**0.5 / Omeg)
    Rs = np.append(Rs, Ps[i] / Is[i]**2)

sigLs = np.array([])
sigRs = np.array([])
for i in range(N):
    print("#", i + 1)
    print(sigRl(Ps[i], Is[i]))
    print(sigL(uls[i], Ps[i], Is[i]))
    sigLs = np.append(sigLs, sigL(uls[i], Ps[i], Is[i]))
    sigRs = np.append(sigRs, sigRl(Ps[i], Is[i]))
    #print("#{}  sigrl: {}  sigL: {}".format(i + 1, sigRl(Ps[i], Is[i]), sigL(uls[i], Ps[i], Is[i]))

sigLs = np.log(Ls) * sigLs / Ls
Ls = np.log(Ls)
sigRs = Rs**(-1) * sigRs / Rs
Rs = 1 / Rs

for i in range(N):
    print("#", i + 1)
    print("1/Rl ", Rs[i], sigRs[i])
    print("ln(L) ", Ls[i], sigLs[i])
xs = np.array([0.5, 0.7, 0.9, 1.1, 1.3, 1.5, 1.7])
middleX = 1.1

#1  rl: 66.0  L: 1.951409746824075
#2  rl: 25.20808561236623  L: 1.7420669178462862
#3  rl: 16.896235078053262  L: 1.5348867445648735
#4  rl: 13.271604938271604  L: 1.3794680492006446
#5  rl: 11.0803324099723  L: 1.2625682834445473
#6  rl: 9.993425378040763  L: 1.1840713591517384
#7  rl: 9.0  L: 1.085172797300043
# 1
3.33766385366
0.0120599223854
# 2
0.901188280106
0.0103283228711
# 3
0.543948852484
0.0102073841722
# 4
0.399652332675
0.0101764671612
# 5
0.322810757042
0.0101580641566
# 6
0.288010517632
0.0101516366411
# 7
0.257390753525
0.0101519754801
# 1
1/Rl  0.0151515151515 0.000766222188626
ln(L)  0.668552058465 0.0041317237186
# 2
1/Rl  0.0396698113208 0.00141819452639
ln(L)  0.55507229207 0.00329089875399
# 3
1/Rl  0.0591847826087 0.0019053649784
ln(L)  0.428456596275 0.00284934448408
# 4
1/Rl  0.0753488372093 0.00226900504461
ln(L)  0.321697953262 0.00237319643545
# 5
1/Rl  0.09025 0.00262931379178
ln(L)  0.233147966601 0.0018758050822
# 6
1/Rl  0.100065789474 0.00288389603498


In [28]:
mpl.rcParams['font.size'] = 20 # Управление стилем, в данном случаем - размером шрифта 
    
# y = b + k

# объекты управления графиком
#fig, ax = plt.subplots()
plt.figure("имя1", figsize=(12, 12))

ax = plt.subplot(221)
# Подписываем оси и график
ax.set_title(r"График №1. Зависимость L и $R_L$ от x - положения сердечника")
ax.set_ylabel(r"$ln(L), ln(Гн)$")
ax.set_xlabel(r"$x, мм$")
#fig.set_figwidth(12)
#fig.set_figheight(12)
# Нанесём точки на график
ax.scatter(xs, Ls)

k = k_coef(xs,Ls)
b = b_coef(xs, Ls)

twoX = np.linspace(0, 2, 2)
twoY = k * twoX + b

middleL = math.exp(k * middleX + b)
sigL = ((math.exp(k * middleX + b)**2 * ( (middleX * k_sigma(xs, Ls))**2 +  b_sigma(x, y)**2))**0.5


ax.errorbar(xs, Ls, yerr=sigLs, xerr=0.05, fmt='.') 
ax.plot(twoX, twoY)#, label=r'T = %f * C + %f'%(k, b), linestyle=mStyle[3])
ax.minorticks_on()
ax.grid(which='major',
        color = 'k', 
        linewidth = 0.5,
       alpha = 0.5)
ax.grid(which='minor', 
        color = 'k', 
        linestyle = ':')



#Второй график

ax = plt.subplot(222)

ax.set_ylabel(r"$1/R, 1/Oм$")
ax.set_xlabel(r"$x, мм$")

ax.scatter(xs, Rs)

k = k_coef(xs, Rs)
b = b_coef(xs, Rs)
middleR = 1 / (k * middleX + b)
sigR = ((1 / (k * middleX + b)**2)**2 * ((middleX * k_sigma(xs, Rs))**2 + b_sigma**2))

twoY = k * twoX + b
ax.plot(twoX, twoY)

ax.errorbar(xs, Rs, yerr=sigRs, xerr=0.05, fmt='.') 
ax.minorticks_on()
ax.grid(which='major',
        color = 'k', 
        linewidth = 0.5,
       alpha = 0.5)
ax.grid(which='minor', 
        color = 'k', 
        linestyle = ':')

mStyle = ['--', '-', '-.', ':']  # - возможные стили линий, строим графики в чб forever

#plt.legend()
#plt.savefig('Зависимость периода от ёмкости.png') # Если нужно сохранить график из кода
plt.show()

SyntaxError: invalid syntax (&lt;ipython-input-28-d2d10935ae6c&gt;, line 29)

NameError: name &#39;middle&#39; is not defined

In [ ]:
print(middleL, sigL)
print(middleR, sigR)